# ex-31 Fit Gumbel Distribution (slower than lmoments)

In [1]:
import numpy as np
import math
from numba import njit
from lmoments3 import distr
import lmoments3 as lm3
from scipy.stats import gumbel_r

from myGEVFit import lmom_ratios as myloms

## Test data

In [2]:
r = gumbel_r.rvs(loc=0.5, scale=2, size=10000)

In [6]:
@njit(fastmath=True)
def newtonRaphsonOptimizer(gumbel_dist, tolerance=0.000000000000001):
    '''
    Function to optimize the multivariate equations using newton raphson
    '''
    s = np.std(gumbel_dist)
    m = np.mean(gumbel_dist)

    #Setting initial values
    a = m - 0.4501 * s
    b = 0.7977 * s

    while True:
        # A Derivative
        a_derivate = 1/b * (len(gumbel_dist) - np.sum(np.exp(-(gumbel_dist-a)/b)))
               
        # B Derivative
        b_derivate = np.sum((gumbel_dist - a)/b**2) - len(gumbel_dist)/b \
                     - np.sum(((gumbel_dist - a)/b**2)*np.exp(-(gumbel_dist - a)/b))
        
        # A Double Derivative
        a_derivate2 = -(1/b**2)*np.sum(np.exp((-(gumbel_dist - a)/b)))
        
        # B Double Derivative
        b_derivate2 = len(gumbel_dist)/b**2 \
                      - (2/b**3) * np.sum((gumbel_dist - a)) \
                      + (2/b**3) * np.sum((gumbel_dist - a) * np.exp(-(gumbel_dist - a)/b)) \
                      - (1/b**4) * np.sum(((gumbel_dist - a)**2) * np.exp(-(gumbel_dist - a)/b))
        
        # AB Derivative
        ab_derivate = -(len(gumbel_dist)/b**2) \
                      + (1/b**2) * np.sum(np.exp(-(gumbel_dist - a)/b)) \
                      - (1/b**3) * np.sum((gumbel_dist - a) * np.exp(-(gumbel_dist - a)/b))
        
        # print(a)
        f = np.array([[a_derivate], [b_derivate]], dtype = np.float64)
        theta = np.sum(f)
        
        h = np.array([[a_derivate2, ab_derivate], [ab_derivate, b_derivate2]], dtype = np.float64)
        h_inv = np.linalg.pinv(h)
        
        x_0 = np.array([[a], [b]], dtype=np.float64)
        x_1 = x_0 - (h_inv * f) 

        t = (x_0[0,0] - x_1[0,0]) * (x_0[0,0] - x_1[0,0]) + (x_0[1,0] - x_1[1,0]) * (x_0[1,0] - x_1[1,0])

        if t <= tolerance:
            break

        a = x_1[0,0] 
        b = x_1[1,0]      

    return a, b

%%time
newtonRaphsonOptimizer(r)

In [36]:
%%timeit 
lm3.lmom_ratios(r, nmom=2)

5.87 ms ± 48.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%%time
distr.gum.fit(r)

Wall time: 3 ms


(0.4651172653002139, 2.0051285822316034)

In [42]:
%%time
gum_lmom_fit(r)

Wall time: 0 ns


[0.46527760507079163, 2.0047719229850247]

In [ ]:
1.3887205594633298/math.log(2)

In [ ]:
1.38872056 /math.log(2)

In [ ]:
distr.gum.lmom_fit(lmom_ratios=lm3.lmom_ratios(r))

In [ ]:
distr.gum.lmom_fit(lmom_ratios=myloms(r))

In [28]:
%%timeit 
myloms(r)

3.23 ms ± 67.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
